# Fraud Detection

### Instalación de librerias.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Carga de dataset.

In [2]:
#fraud_det_1 = pd.read_csv("C:/Users/juacevedo.ATLANTICQI/Downloads/train_identity.csv")
#fraud_det_2 = pd.read_csv("C:/Users/juacevedo.ATLANTICQI/Downloads/train_transaction.csv")
#fraud_det=pd.merge(fraud_det_1, fraud_det_2, on='TransactionID', how='left')
fraud_det = pd.read_csv("C:/Users/juacevedo.ATLANTICQI/Downloads/train_transaction.csv")

## EDA

### Previsualizacion de dataset

In [3]:
fraud_det.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
fraud_det.tail()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
590535,3577535,0,15811047,49.00,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590539,3577539,0,15811131,279.95,W,15066,170.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Forma del data set

In [8]:
fraud_det.shape

(590540, 394)

El dataset tiene 590.540 filas y 394 columnas

### Identificaremos el tipo de datos de cada variable.

In [5]:
fraud_det.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 394 entries, TransactionID to V339
dtypes: float64(376), int64(4), object(14)
memory usage: 1.7+ GB


El dataset presenta:

376 columnas de tipo float.

14 columnas de tipo categoricas.

4 columnas de tipo entero.

### Comprobación de duplicados

In [18]:
fraud_det.nunique()

TransactionID     590540
isFraud                2
TransactionDT     573349
TransactionAmt     20902
ProductCD              5
                   ...  
V335                 672
V336                 356
V337                 254
V338                 380
V339                 334
Length: 394, dtype: int64

Si una columna tiene un número muy bajo de valores únicos en comparación con el número total de filas, podría indicar que es una variable categórica o que tiene muchos valores repetidos.
Si una columna tiene un número de valores únicos cercano al número total de filas, es probable que sea una variable continua o un identificador único.

Ejemplo:

TransactionID:

Tiene 590.540 valores únicos. Esto sugiere que cada TransactionID es único para cada fila, lo que tiene sentido ya que es un identificador único para cada transacción.

isFraud:

Tiene 2 valores únicos. Esto significa que en la columna 2 valores diferentes 1 o 0.

### Conteo de datos faltantes y porcentaje

In [13]:
fraud_det.isnull().sum()

TransactionID        0
id_01                0
id_02             3361
id_03            77909
id_04            77909
                 ...  
V335             62192
V336             62192
V337             62192
V338             62192
V339             62192
Length: 434, dtype: int64

In [14]:
(fraud_det.isnull().sum()/(len(fraud_det)))*100

TransactionID     0.000000
id_01             0.000000
id_02             2.330257
id_03            54.016071
id_04            54.016071
                   ...    
V335             43.119120
V336             43.119120
V337             43.119120
V338             43.119120
V339             43.119120
Length: 434, dtype: float64

In [15]:
import pandas as pd
# Calcula el número de datos nulos en cada columna
null_counts = fraud_det.isnull().sum()

# Determina cuántas columnas tienen al menos un dato nulo
columns_with_nulls = null_counts[null_counts > 0].shape[0]

print(f"El DataFrame tiene {columns_with_nulls} columnas con datos nulos.")


El DataFrame tiene 412 columnas con datos nulos.


### Identificar variables categoricas y numericas.

In [27]:
import numpy as np
cat_cols=fraud_det.select_dtypes(include=['object']).columns
num_cols = fraud_det.select_dtypes(include=np.number).columns.tolist()
print("Categorical Variables:")
print(cat_cols)
print("Numerical Variables:")
print(num_cols)

Categorical Variables:
Index(['id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo', 'ProductCD', 'card4', 'card6',
       'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6',
       'M7', 'M8', 'M9'],
      dtype='object')
Numerical Variables:
['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_13', 'id_14', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_24', 'id_25', 'id_26', 'id_32', 'isFraud', 'TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14',

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
encoded_data = encoder.fit_transform(fraud_det[['id_12']])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['id_12']))
encoded_df


In [18]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Instanciar el OneHotEncoder
encoder = OneHotEncoder(sparse=False, drop='first')

# Ajustar y transformar las columnas categóricas del dataframe
encoded_data = encoder.fit_transform(fraud_det[cat_cols])

# Convertir los datos codificados en un dataframe y agregar nombres de columnas
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(cat_cols))

# Concatenar el dataframe original (sin las columnas categóricas) con el dataframe codificado
fraud_det_encoded = pd.concat([fraud_det.drop(columns=cat_cols), encoded_df], axis=1)
fraud_det_encoded.head()


c:\Users\juacevedo.ATLANTICQI\AppData\Local\miniconda3\envs\ml_venv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,R_emaildomain_yahoo.com,R_emaildomain_yahoo.com.mx,R_emaildomain_yahoo.de,R_emaildomain_yahoo.es,R_emaildomain_yahoo.fr,R_emaildomain_ymail.com,R_emaildomain_nan,M4_M1,M4_M2,M4_nan
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [26]:
import numpy as np
cat_cols=fraud_det_encoded.select_dtypes(include=['object']).columns
num_cols = fraud_det_encoded.select_dtypes(include=np.number).columns.tolist()
print("Categorical Variables:")
print(cat_cols)
print("Numerical Variables:")
print(num_cols)

Categorical Variables:
Index([], dtype='object')
Numerical Variables:
['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_13', 'id_14', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_24', 'id_25', 'id_26', 'id_32', 'isFraud', 'TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59'

### Conteo de datos faltantes y porcentaje

In [20]:
fraud_det_encoded.isnull().sum()

TransactionID                  0
id_01                          0
id_02                       3361
id_03                      77909
id_04                      77909
                           ...  
R_emaildomain_ymail.com        0
R_emaildomain_nan              0
M4_M1                          0
M4_M2                          0
M4_nan                         0
Length: 2815, dtype: int64

In [21]:
(fraud_det_encoded.isnull().sum()/(len(fraud_det_encoded)))*100

TransactionID               0.000000
id_01                       0.000000
id_02                       2.330257
id_03                      54.016071
id_04                      54.016071
                             ...    
R_emaildomain_ymail.com     0.000000
R_emaildomain_nan           0.000000
M4_M1                       0.000000
M4_M2                       0.000000
M4_nan                      0.000000
Length: 2815, dtype: float64

In [22]:
import pandas as pd
# Calcula el número de datos nulos en cada columna
null_counts = fraud_det_encoded.isnull().sum()

# Determina cuántas columnas tienen al menos un dato nulo
columns_with_nulls = null_counts[null_counts > 0].shape[0]

print(f"El DataFrame tiene {columns_with_nulls} columnas con datos nulos.")


El DataFrame tiene 383 columnas con datos nulos.


In [23]:
import pandas as pd
# Encuentra las columnas que tienen al menos un dato nulo
columns_with_nulls = fraud_det_encoded.columns[fraud_det_encoded.isnull().any()].tolist()

print("Las columnas con datos nulos son:", columns_with_nulls)


Las columnas con datos nulos son: ['id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_13', 'id_14', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_24', 'id_25', 'id_26', 'id_32', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', '

In [24]:
(fraud_det_encoded.isnull().sum()/(len(fraud_det_encoded)))*100

TransactionID               0.000000
id_01                       0.000000
id_02                       2.330257
id_03                      54.016071
id_04                      54.016071
                             ...    
R_emaildomain_ymail.com     0.000000
R_emaildomain_nan           0.000000
M4_M1                       0.000000
M4_M2                       0.000000
M4_nan                      0.000000
Length: 2815, dtype: float64

In [25]:
import pandas as pd
from sklearn.impute import SimpleImputer
fraud_det_encoded[column] = imputer.fit_transform(fraud_det_encoded[[column]])

NameError: name 'imputer' is not defined

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Suponiendo que fraud_det_encoded es tu DataFrame
imputer = SimpleImputer(strategy='median')

for column in fraud_det_encoded.columns:
    # Verifica si la columna tiene datos faltantes
    if fraud_det_encoded[column].isnull().sum() > 0:
        # Imputa los datos nulos con la mediana para esa columna específica
        fraud_det_encoded[column] = imputer.fit_transform(fraud_det_encoded[[column]])

# Ahora, fraud_det_encoded tiene los valores nulos imputados con la mediana en las columnas que tenían datos faltantes



IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
fraud_det_encoded.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,R_emaildomain_yahoo.com,R_emaildomain_yahoo.com.mx,R_emaildomain_yahoo.de,R_emaildomain_yahoo.es,R_emaildomain_yahoo.fr,R_emaildomain_ymail.com,R_emaildomain_nan,M4_M1,M4_M2,M4_nan
0,2987004,0.0,70787.0,0.0,0.0,0.0,0.0,14.0,-34.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,2987008,-5.0,98945.0,0.0,0.0,0.0,-5.0,14.0,-34.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,14.0,-34.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2987011,-5.0,221832.0,0.0,0.0,0.0,-6.0,14.0,-34.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,14.0,-34.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [ ]:
fraud_det_encoded.columns

Index(['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06',
       'id_07', 'id_08', 'id_09',
       ...
       'R_emaildomain_yahoo.com', 'R_emaildomain_yahoo.com.mx',
       'R_emaildomain_yahoo.de', 'R_emaildomain_yahoo.es',
       'R_emaildomain_yahoo.fr', 'R_emaildomain_ymail.com',
       'R_emaildomain_nan', 'M4_M1', 'M4_M2', 'M4_nan'],
      dtype='object', length=2815)

In [ ]:
# Convertir el índice de columnas a una lista
column_names = fraud_det_encoded.columns.tolist()

# Imprimir la lista de nombres de columnas
for name in column_names:
    print(name)

# Imprimir el total de columnas
print("\nTotal de columnas:", len(column_names))


TransactionID
id_01
id_02
id_03
id_04
id_05
id_06
id_07
id_08
id_09
id_10
id_11
id_13
id_14
id_17
id_18
id_19
id_20
id_21
id_22
id_24
id_25
id_26
id_32
isFraud
TransactionDT
TransactionAmt
card1
card2
card3
card5
addr1
addr2
dist1
dist2
C1
C2
C3
C4
C5
C6
C7
C8
C9
C10
C11
C12
C13
C14
D1
D2
D3
D4
D5
D6
D7
D8
D9
D10
D11
D12
D13
D14
D15
V1
V2
V3
V4
V5
V6
V7
V8
V9
V10
V11
V12
V13
V14
V15
V16
V17
V18
V19
V20
V21
V22
V23
V24
V25
V26
V27
V28
V29
V30
V31
V32
V33
V34
V35
V36
V37
V38
V39
V40
V41
V42
V43
V44
V45
V46
V47
V48
V49
V50
V51
V52
V53
V54
V55
V56
V57
V58
V59
V60
V61
V62
V63
V64
V65
V66
V67
V68
V69
V70
V71
V72
V73
V74
V75
V76
V77
V78
V79
V80
V81
V82
V83
V84
V85
V86
V87
V88
V89
V90
V91
V92
V93
V94
V95
V96
V97
V98
V99
V100
V101
V102
V103
V104
V105
V106
V107
V108
V109
V110
V111
V112
V113
V114
V115
V116
V117
V118
V119
V120
V121
V122
V123
V124
V125
V126
V127
V128
V129
V130
V131
V132
V133
V134
V135
V136
V137
V138
V139
V140
V141
V142
V143
V144
V145
V146
V147
V148
V149
V150
V151
V152
V153
V154
V15

In [ ]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

data = fraud_det_encoded.drop(['TransactionID',
                               'isFraud'],axis=1)

vif_data = pd.DataFrame()
vif_data["Variable"] = data.columns
vif_data["VIF"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
vif_data


c:\Users\juacevedo.ATLANTICQI\AppData\Local\miniconda3\envs\ml_venv\lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


KeyboardInterrupt: 